# MyoSDK Tutorial

## What is MyoSDK?

MyoSapiens SDK (MyoSDK) is a Python library that allows you to **Retarget motion** from marker datas onto a 3D character.

## What You'll Learn

This tutorial will walk you through the complete process:
1. Setting up the MyoSDK client
2. Apply marker data describing a motion to a 3D character (retargeting)
3. Downloading the final animated character file

## Prerequisites

Before starting, make sure you have:
- **Python 3.8 or higher** installed
- **An API key** from MyoLab (you'll need to sign up and get your API key)
- **A Mocap file in c3d format** showing a person moving
- **A Markerset file** describing approximately where the markers are placed on the body
- **Basic Python knowledge** (understanding variables, functions, and basic syntax)

## Installation

If you haven't installed `myosdk` yet, run this command in your terminal:
```bash
pip install myosdk
```

In [ ]:
# Step 1: Import required libraries
import os
import myosdk
from myosdk import Client
# Step 2: Set up your API key
# The API key is like a password that authenticates you with the MyoSDK service.
# You can get your API key from your MyoLab account dashboard at dev.myolab.ai.
# Instructions are available at https://docs.myolab.ai/docs/myosdk/getting-started/api-key

# Option 1 (Recommended): Set it as an environment variable
# In your terminal, run: export MYOSDK_API_KEY="your-api-key-here"
# Or on Windows: set MYOSDK_API_KEY=your-api-key-here

# Option 2: Replace "..." below with your actual API key (less secure, but easier for testing)
api_key = os.getenv("MYOSDK_API_KEY", "...")

# Step 3: Create the client
# The client is your connection to the MyoSDK service. It handles all communication
# with the servers. The base_url tells it which server to connect to.
client = Client(api_key=api_key)

# Verify the connection worked
print("Client initialized successfully.")
print(f"✓ You're ready to use MyoSDK version {myosdk.__version__}!")

## 1. Process video and upload Input Assets

We need to process a video. We used Metrabs, a non-commercial OSS video to motion library, to produce 3d trackers. This and the corresponding markerset files are then upload

In [ ]:
from metrabs_pytorch.scripts.run_video import run_metrabs_video
from myo_tools.mjs.marker.marker_api import get_marker_names
from myo_tools.utils.file_ops.dataframe_utils import from_array_to_dataframe
import torch
import numpy as np
from myo_tools.utils.mocap_ops.mocap_utils import rotate_mocap_ydown_to_zup

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

video_path = "../data/13710671_1080_1920_25fps.mp4"
results = list(
            run_metrabs_video(video_path=video_path, device=DEVICE, visualize=False)
        )

# trackers from videos are in opencv space i.e. y-up, and we need in z-up space
markers = rotate_mocap_ydown_to_zup(np.array([res["poses3d"] for res in results]).squeeze())

fps = results[0]["fps"]

markerset_path = "../markersets/movi_metrabs_markerset.xml"

# Upload trackers file
marker_names = get_marker_names(markerset_path)
fn_parquet = "video_trackers.parquet"
from_array_to_dataframe(markers, marker_names, fps, fn_parquet)
tracker_asset = client.assets.upload_file(fn_parquet, purpose="retarget")

print(tracker_asset)

# Upload markerset XML file
markerset_asset = client.assets.upload_file(markerset_path)
print(markerset_asset)


## 2. Motion Retargeting

Use the extracted motion data (trackers) to retarget onto a 3D character. This is called "retargeting."

**What is retargeting?** Think of it like this: you recorded someone dancing, extracted their dance moves, and now you're applying those exact same moves to a 3D character. The character will move in the same way as the person in your video.

**What you need:**
- The `tracker_asset` describing the motion data
- The `markerset_asset` describing the association of motion data to body parts


In [ ]:
# Start retargeting job
job = client.jobs.start_retarget(
    tracker_asset_id=tracker_asset["asset_id"],
    markerset_asset_id=markerset_asset["asset_id"],
)

# Wait for job to complete
result = client.jobs.wait(job["job_id"])
asset_ids = result["output"]["retarget_output_asset_ids"]
print(asset_ids)

## 3. Download Result

Finally, we'll download the joint angles.

**Note:** Make sure the output directory exists, or the download will create it for you.

In [ ]:
output_parquet_path = "tmp-assets/tutorial_output.parquet"

print(f"Downloading motion joint angles result to {output_parquet_path}...")
os.makedirs(os.path.dirname(output_parquet_path), exist_ok=True)
client.assets.download(asset_ids["qpos"], output_parquet_path)

print("Download complete!")

In [ ]:
from myo_tools.utils.file_ops.io_utils import from_qpos_to_joint_angles
# Reads a dataframe with qpos and converts it to joint angles
data = from_qpos_to_joint_angles(output_parquet_path)
data.head()